# Text preprocessing, N-grams and language models - Tianyou He

This exercise will use the “SIGNAL NEWS1” corpus which provides news stories formatted in JSON. Each line contains a JSON item with a news story. 
I first begin with importing all the necessary packages, followed by retrieving each news story from the “content” field of each JSON element in the corpus. I then lowercase them and place them in a list called **data**.

In [19]:
# Import packages
import json
import re
from nltk import word_tokenize, FreqDist, bigrams, trigrams, ngrams
from nltk.stem import WordNetLemmatizer
from collections import Counter, defaultdict
import math

# Read and lowercase text
data = []
for line in open('signal-news1/signal-news1.jsonl', 'r'):
    data.append(json.loads(line))
for i in range(0,len(data)):
    data[i] = str(data[i]['content']).lower()

## Text preprocessing
Text preprocessing is conducted in four parts using regular expressions. Iterating across each news story in **data**, I first remove URLs, followed by numbers that are fully made of digits, then I remove all non-alphanumeric characters besides spaces, and finally remove words with only one character. The reader will be right to notice that given the different forms appearing in URLs, it would be inappropriate to remove non-alphanumeric characters prior to removing URLs. For example, removing special characters such as "://" or the "." in ".com" will result in the failure of the URL regular expression to capture the whole URL afterwards for removal. The reader will also be right in noticing the same rationale for removing just numbers prior, since we wish to preserve certain strings such as those representing time in an HH:MM format. Removing ":" prior would have resulted in the removal of "HH" and "MM" after.

In [20]:
for i in range(0,len(data)):
# Remove URLs. Note that URLs may appear in different forms, e.g. “http://www.*”, “http://domain”, “https://www.*”
    data[i] = re.sub(r'(((https?:\/\/)|(w{3}.))[\S]*)|([\w\d\/\.]*\.(com|cn|co|net|org|edu|uk|int|js|html))','',data[i])
# Remove numbers that are fully made of digits (e.g. the number ‘5’, but keep the case of ‘5pm’)
    data[i] = re.sub(r'\b\d+\b','',data[i])
# Remove all non-alphanumeric characters except spaces
    data[i] = re.sub(r'[^a-z0-9\s]','', data[i])
# Remove words with only 1 character
    data[i] = re.sub(r'\b\w\b','',data[i])

In the final stage of text preprocessing, using list comprehensions I tokenize and also lemmatize each story for usage later. Because **WordNetLemmatizer()** operates on a single token, I use nested list comprehensions to compute each lemma. These tokens and lemmas are stored individually in lists of list named **tokenized_data** and **lemmatized_data** respectively. Both lists preserves the index of each story in the corpus.

In [21]:
# Process all the words using an English lemmatiser
lemma = WordNetLemmatizer()
tokenized_data = [word_tokenize(story) for story in data]
lemmatized_data = [[lemma.lemmatize(token) for token in story] for story in tokenized_data]

## N-grams
Lets see what we are working with. To compute the number of tokens **N** in our corpus, I first calculate the length of each story (list of tokens) before summing up these lengths. To compute the size of the vocabulary of unigrams **V**, I begin by flattening out the list **lemmatized_data** in order to allow the use of the **set()** function to return only a list of unique elements. I then simply compute the length of that list to compute vocabulary size **V**.

In [22]:
# Compute N (number of tokens) and V (vocabulary size)
print("Number of tokens N: ", sum([len(i) for i in tokenized_data]))
data_lemmas = [vocab for story in lemmatized_data for vocab in story]
print("Vocabulary size V: ", len(set(data_lemmas)))

Number of tokens N:  5692062
Vocabulary size V:  120001


Now, going through the flattened list **data_lemmas** created earlier. I construct their trigrams using **trigrams()** before computing their frequency distribution. I then access and list the top 25 occurring trigrams using the **most_common()** object from the frequency distribution class generated earlier. Results are shown below.

In [23]:
# The top 25 trigrams based on the number of occurrences on the entire corpus
trigrams = trigrams(data_lemmas)
trigram_freq = FreqDist(trigrams)
print("--- Top 25 Trigrams ---")
print(trigram_freq.most_common(25))

--- Top 25 Trigrams ---
[(('one', 'of', 'the'), 2434), (('on', 'share', 'of'), 2095), (('day', 'moving', 'average'), 1979), (('on', 'the', 'stock'), 1567), (('a', 'well', 'a'), 1424), (('in', 'research', 'report'), 1415), (('in', 'research', 'note'), 1373), (('the', 'united', 'state'), 1223), (('for', 'the', 'quarter'), 1221), (('average', 'price', 'of'), 1193), (('research', 'report', 'on'), 1177), (('research', 'note', 'on'), 1138), (('share', 'of', 'the'), 1132), (('the', 'end', 'of'), 1130), (('in', 'report', 'on'), 1124), (('earnings', 'per', 'share'), 1121), (('cell', 'phone', 'plan'), 1073), (('phone', 'plan', 'detail'), 1070), (('according', 'to', 'the'), 1066), (('of', 'the', 'company'), 1058), (('buy', 'rating', 'to'), 1016), (('appeared', 'first', 'on'), 995), (('moving', 'average', 'price'), 995), (('price', 'target', 'on'), 981), (('part', 'of', 'the'), 935)]


I now want to conduct sentiment analysis. Lets start by counting the number of occurrences for postive and negative words across the corpus. To do so, I begin by initializing two lists, a list of positive words and another of negative words

In [24]:
# Create lists of positive and negative words
with open('signal-news1/opinion-lexicon-English/positive-words.txt', 'r') as pwords:
    pos_words = pwords.read().splitlines()
with open('signal-news1/opinion-lexicon-English/negative-words.txt', 'r') as nwords:
    neg_words = nwords.read().splitlines()

With the created lists above, I now run through each word in every news story of **tokenized_data** and compare that word to the lists of positive and negative words. If that word is either a positive or negative word it will increment its respective count by one. The total of both positive and negative words within each news story will then be appended to their respective list **pos_word_count** and **neg_word_count**, where their index will be equivalent to the index of the news story the words originated from. This is done so to initialize the values to answer the next question more efficiently. 

Given the brute search nature of this algorithm, this code chunk will take about 5 minutes to execute. The resulting number of positive words and negative words are shown below. It would appear that the whole corpus tend to have a more positive sentiment.

In [25]:
# Compute the number of positive and negative word counts in the corpus
pos_word_count = []
neg_word_count = []
for story in tokenized_data:
    pos_count = 0
    neg_count = 0
    for word in story:
        if word in pos_words:
            pos_count += 1
        elif word in neg_words:
            neg_count += 1
        else:
            continue
    pos_word_count.append(pos_count)
    neg_word_count.append(neg_count)
print("Number of positive words in corpus: ", sum(pos_word_count))
print("Number of negative words in corpus: ", sum(neg_word_count))

Number of positive words in corpus:  164916
Number of negative words in corpus:  122651


Now we want to know the number of stories that has more positive words and also those that has more negative words. Fortunately, we have already counted the number of positive and negative words in each story, storing their values in **pos_word_count** and **neg_word_count**. The algorithm below simply runs through every index in the corpus and compares those stored values of positive and negative words for each indexed story. 

If there are more positive words in that story then **more_pos_stories** is incremented by one, vice versa for **more_neg_stories**. For this exercise, neutral stories where there is an equal number of positive and negative words - will be ignored. From the results shown below, we observe that there are more positive stories than negative stories.

In [26]:
# Compute the number of news stories with more positive than negative words and those with with more negative than positive words
more_pos_stories = 0
more_neg_stories = 0
for i in range(len(tokenized_data)):
    if pos_word_count[i] > neg_word_count[i]:
        more_pos_stories += 1
    elif pos_word_count[i] < neg_word_count[i]:
        more_neg_stories += 1
    else:
        continue
print("Number of stories with more positive words: ", more_pos_stories)
print("Number of stories with more negative words: ", more_neg_stories)

Number of stories with more positive words:  10847
Number of stories with more negative words:  6301


## Language models
Now I will build a trigram language model from the first 16,000 stories in the corpus, I will then compute the perplexity of that model by testing it on the remaining 16,001+ stories in the corpus. For dealing with unorbserved occurences in the training data that occurs in the test data, I will use general laplace smoothing with the **k** parameter set to 0.001. 

The chosen value 0.001 is an arbitrary decision to constrain the amount of probability mass to be transfered from the "good" probabilities I have in my model to the unseen trigrams present in the test data. I wish to explore some of the more sophisticated approaches such as Backoff or Kneser-Ney smoothing in the future, but I will experiment with laplace smoothing for now.

Before constructing the model, I will first compute the vocabulary size of bigrams to be used later in laplace smoothing. From the result below we can see that the number of unique bigrams in the corpus is 1,577,545. Given this large number it would be justified that setting **k** = 0.001 was a sensible decision.

In [27]:
bigrams = bigrams(data_lemmas)
bigram_freq = FreqDist(bigrams)
unique_bigrams = len(bigram_freq)
print("Number of unique bigrams: ", unique_bigrams)

Number of unique bigrams:  1577545


Now to construct the model we create a dictionary which stores the frequencies each trigram combination occurs in the first 16,000 stories in the corpus. We also convert these frequencies into the smoothed probabilities given the previous two words.

In [28]:
# Compute language models for trigrams based on the first 16,000 rows of the corpus.
k = 0.001
model = defaultdict(lambda: defaultdict(lambda: 0))
for story in tokenized_data[0:16000]:
    for w1, w2, w3 in ngrams(story, 3):
        model[(w1, w2)][w3] += 1
for w1_w2 in model:
    total_count = float(sum(model[w1_w2].values()))
    for w3 in model[w1_w2]:
        model[w1_w2][w3] += k 
        model[w1_w2][w3] /= (total_count + k * unique_bigrams)

With our newly created model we can now access next word probabilities for each pair of words. Below I show an example of the top five most probable words following the words "is" and "the".

In [29]:
sorted(dict(model["is", "this"]).items(), key=lambda kv: kv[1], reverse=True)[0:5]

[('the', 0.002954722031024286),
 ('going', 0.0023638957900676202),
 ('one', 0.0017730695491109541),
 ('woman', 0.0017730695491109541),
 ('good', 0.0017730695491109541)]

Lets now predict a sentence of ten words where each word is the most likely word to follow the previous two words. Beginning with the words **"is this"**, the resulting prediction as seen below is **"is this the company has capitalization of billion and pe ratio"**.

In [33]:
sentence = ['is', 'this']
while len(sentence) < 12:
    word_1 = sentence[-2]
    word_2 = sentence[-1]
    likely_next_word = sorted(dict(model[word_1, word_2]).items(), key=lambda kv: kv[1], reverse=True)[0][0]
    sentence.append(likely_next_word)
else:
    print(sentence)

['is', 'this', 'the', 'company', 'has', 'market', 'capitalization', 'of', 'billion', 'and', 'pe', 'ratio']


Lets generate another sentence just for fun. Beginning with the words **"time to"**, the resulting prediction as seen below is **"time to get the latest news and analysts ratings for tsla"**. It is interesting to see that the ticker symbol "tsla" for Tesla is the most popular word in the corpus following the words "ratings for".

In [38]:
sentence = ['time', 'to']
while len(sentence) < 11:
    word_1 = sentence[-2]
    word_2 = sentence[-1]
    likely_next_word = sorted(dict(model[word_1, word_2]).items(), key=lambda kv: kv[1], reverse=True)[0][0]
    sentence.append(likely_next_word)
else:
    print(sentence)

['time', 'to', 'get', 'the', 'latest', 'news', 'and', 'analysts', 'ratings', 'for', 'tsla']


To see how well our model performs, perplexity needs to be computed as a measure. A lower perplexity will imply a better model. The perplexity will be computed over the remaining 16,001+ stories in the corpus not involved in training the model. I begin by iterating through each trigram in the remaining corpus and then append their smoothed probabilities to the list **probs**. If the trigram encountered was unseen in the training data, I simply append the laplace smoothed probability derived from the equation.

In [35]:
# Computing perplexity by evaluating on the remaining rows of the corpus (rows 16,001+)
probs = []
for story in tokenized_data[16001:len(tokenized_data)]:
    for w1, w2, w3 in ngrams(story, 3):
        if model[w1,w2][w3] > 0:
            probs.append(model[w1,w2][w3])
        else:
            total_count = float(sum(model[w1_w2].values()))
            smoothed_prob = k / (total_count + k * unique_bigrams)
            probs.append(smoothed_prob)

Due to the inpracticality of computing the product over the large amount of values, I will use the Shannon entropy equivalent of the formula where I can simply sum the logs (of base 2) of each laplace smoothed probability, before averaging it to get its entropy. Then, to compute the perplexity I simply raise 2 to the power of the entropy just calculated. The resulting perplexity of the trigram model over the test data is then 59,016.

In [36]:
n = len(probs)
log_probs = [math.log2(prob) for prob in probs]
log_probs_sum = sum(log_probs)
perplexity = 2**(-log_probs_sum/n)
print("Perplexity: ", perplexity)

Perplexity:  59015.726783455044


The model appears to have a pretty high perplexity, this is an artifact from the large number of **unique_bigrams** used in the deonminator of the laplace smoothing process. I will explore other smoothing techniques in the future and see how they can improve the model.